In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from pathlib import Path
from argparse import Namespace
from collections import defaultdict

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

import xgboost as xgb

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                #if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #    df[col] = df[col].astype(np.float16)
                #elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                #    df[col] = df[col].astype(np.float32)
                #else:
                df[col] = df[col].astype(np.float16)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:

#train = pd.read_csv('../input/ubiquant-market-prediction/train.csv')
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

In [ ]:
#for i in range(0,l-1):
#    for j in range(4,303):
#        train.iloc[i,j]=round(train.iloc[i,j],6)
#train=train.round(10)
#train.head(5)

In [ ]:

train = reduce_mem_usage(train)

trains = train.dropna(how="any")


# model goes here
#1st model


In [ ]:
l=2200
train=trains[trains['investment_id']<=l]
test=trains[(trains['investment_id']>l) & (trains['investment_id']<l+20)]
train['investment_id'].nunique()

In [ ]:
import gc
gc.collect()
test_cols = ['investment_id','row_id']
features = [c for c in train.columns if "f_" in c]
features = test_cols + features

X_train = train.loc[:, features]
    
y_train = train['target']

import gc
gc.collect()
cdf=X_train[:int(len(X_train)/50)].corr()

exla={}
exl=[]
inl=[]
topcorr=0
topidx=''
count=0
for i in range(1,300):
    topcorr=0
    topidx=''
    for j in range(1,300):
        if(i!=j):
            if(cdf.index[i] not in exl and cdf.index[j] not in exl):
                if(cdf.iloc[i,j]>0.7):
                    exl.append(cdf.index[j])
                    topcorr=0
                    topidx=''
                if(cdf.iloc[i,j]>0.5 and cdf.index[j] not in exl):
                    if(topcorr<cdf.iloc[i,j]):
                        topcorr=cdf.iloc[i,j]
                        topidx=cdf.index[j]
    if(topcorr>0):
        X_train[cdf.index[i]]=(X_train[cdf.index[i]]+X_train[topidx])/2
        exl.append(topidx)
        exla[count,count]=cdf.index[i]
        exla[count,count+1]=topidx
        count=count+1
                    



In [ ]:
df=X_train
df['row_id']=((df['row_id']).str.strip())
#df['len']=pd.to_numeric((df['row_id']).str.len()-2)
df['row_id']=df['row_id'].apply(lambda x:x[0:x.find('_')])
df['row_id']=pd.to_numeric(df['row_id'])
df

In [ ]:
X_train=df

In [ ]:
import gc
gc.collect()
from sklearn.model_selection import train_test_split
#X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=1)
X_train=X_train
y_train=y_train
X_test = test.loc[:, features]

X_test['row_id']=((X_test['row_id']).str.strip())
X_test['row_id']=X_test['row_id'].apply(lambda x:x[0:x.find('_')])
X_test['row_id']=pd.to_numeric(X_test['row_id'])

y_test = test['target']
X_train.head(5)

In [ ]:
model = xgb.XGBRegressor(
    n_estimators=800,
    learning_rate=0.03,
    max_depth=12,
    subsample=0.95,
    colsample_bytree=0.9,
    #colsample_bylevel=0.75,
    missing=-999,
    random_state=1,
    tree_method='gpu_hist'  
    )
    
import gc
gc.collect()
    
model.fit(X_train, y_train, early_stopping_rounds=20, eval_set=[(X_test, y_test)], verbose=1)



In [ ]:
import gc
gc.collect()
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()



In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    
    x_tt = test_df.loc[:, features]
    #print(x_tt.head(2))
    x_tt['row_id']=((x_tt['row_id']).str.strip())
    x_tt['row_id']=x_tt['row_id'].apply(lambda x:x[0:x.find('_')])
    x_tt['row_id']=pd.to_numeric(x_tt['row_id'])
    x_tt=x_tt.values
    
    #y_pred = model.predict(cudf.from_pandas(pd.DataFrame([x_test])).astype('float32'))
    #y_pred = model.predict(pd.DataFrame([x_test]))[0]
    xgb_pred = model.predict(x_tt)
    sample_prediction_df['target'] = xgb_pred
    env.predict(sample_prediction_df) 
    display(sample_prediction_df)